In [1]:
import folium
import json
import pandas as pd
import numpy as np
import geopandas as gpd
import fiona
import os
import seaborn as sns
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, LineString, Point
from tqdm import tqdm

In [2]:
data = gpd.read_file('data/서울시 격자 인구 데이터/강남서초2030대인구수 - 복사본.geojson')
data

,gid,2030val,layer,geometry
0,다사579454,77.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.02366 37.50691, 127.02365 ..."
1,다사613448,66.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.06216 37.50166, 127.06215 ..."
2,다사585452,0.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.03046 37.50514, 127.03045 ..."
3,다사583456,227.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.02817 37.50874, 127.02817 ..."
4,다사592432,85.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.03849 37.48714, 127.03848 ..."
...,...,...,...,...
9128,다사559438,NaN,"서초구 20,30대 인구수","MULTIPOLYGON (((127.00113 37.49240, 127.00112 ..."
9129,다사618387,NaN,"서초구 20,30대 인구수","MULTIPOLYGON (((127.06813 37.44669, 127.06813 ..."
9130,다사559464,NaN,"서초구 20,30대 인구수","MULTIPOLYGON (((127.00097 37.51583, 127.00097 ..."
9131,다사585417,NaN,"서초구 20,30대 인구수","MULTIPOLYGON (((127.03066 37.47359, 127.03065 ..."


In [5]:
data.loc[data['2030val'].apply(pd.isna), '2030val'] = 0
data

,gid,2030val,layer,geometry
0,다사579454,77.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.02366 37.50691, 127.02365 ..."
1,다사613448,66.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.06216 37.50166, 127.06215 ..."
2,다사585452,0.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.03046 37.50514, 127.03045 ..."
3,다사583456,227.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.02817 37.50874, 127.02817 ..."
4,다사592432,85.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.03849 37.48714, 127.03848 ..."
...,...,...,...,...
9128,다사559438,0.0,"서초구 20,30대 인구수","MULTIPOLYGON (((127.00113 37.49240, 127.00112 ..."
9129,다사618387,0.0,"서초구 20,30대 인구수","MULTIPOLYGON (((127.06813 37.44669, 127.06813 ..."
9130,다사559464,0.0,"서초구 20,30대 인구수","MULTIPOLYGON (((127.00097 37.51583, 127.00097 ..."
9131,다사585417,0.0,"서초구 20,30대 인구수","MULTIPOLYGON (((127.03066 37.47359, 127.03065 ..."


In [6]:
data_gid = data[['gid', 'geometry']]

In [7]:
data_gid

,gid,geometry
0,다사579454,"MULTIPOLYGON (((127.02366 37.50691, 127.02365 ..."
1,다사613448,"MULTIPOLYGON (((127.06216 37.50166, 127.06215 ..."
2,다사585452,"MULTIPOLYGON (((127.03046 37.50514, 127.03045 ..."
3,다사583456,"MULTIPOLYGON (((127.02817 37.50874, 127.02817 ..."
4,다사592432,"MULTIPOLYGON (((127.03849 37.48714, 127.03848 ..."
...,...,...
9128,다사559438,"MULTIPOLYGON (((127.00113 37.49240, 127.00112 ..."
9129,다사618387,"MULTIPOLYGON (((127.06813 37.44669, 127.06813 ..."
9130,다사559464,"MULTIPOLYGON (((127.00097 37.51583, 127.00097 ..."
9131,다사585417,"MULTIPOLYGON (((127.03066 37.47359, 127.03065 ..."


## 강남구서초구 출근 자전거

In [ ]:
data1 = pd.read_csv("output/강남구서초구정류장아침승차.csv")

In [ ]:
data1 = data1[["표준ID","정류소명","X좌표","Y좌표","6시승차총승객수","7시승차총승객수","8시승차총승객수","9시승차총승객수"]]

In [ ]:
data1["아침총승객수"] = data1["6시승차총승객수"] + data1["7시승차총승객수"]+ data1["8시승차총승객수"]+ data1["9시승차총승객수"]

In [ ]:
data1 = data1[["표준ID","정류소명","X좌표","Y좌표","아침총승객수"]]
data1

In [ ]:
data1['bus_cnt']=1
data1['X좌표'] = data1['X좌표'].astype(float)
data1['Y좌표'] = data1['Y좌표'].astype(float)
data1['point'] = data1.apply(lambda row : Point([row['X좌표'], row['Y좌표']]), axis=1)
data1 = gpd.GeoDataFrame(data1, geometry='point')

In [ ]:
data1

In [ ]:
gid_list = []
for i in tqdm(range(data1.shape[0])):
    point = data1['point'].iloc[i]
    if data_gid['geometry'].contains(point).sum() == 0:
        g1 = "Na"
    else :
        g = data_gid['gid'].loc[data_gid['geometry'].contains(point)]
        g1 = g.iloc[0]
    gid_list.append(g1)

In [ ]:
data1['gid'] = gid_list
data1['TL_cnt']=1
data1

In [ ]:
data1 = data1[["gid","아침총승객수"]]
data1

In [ ]:
result1 = data1[['아침총승객수', 'gid']].groupby(['gid']).sum()

In [ ]:
result1.reset_index(inplace=True)

In [ ]:
result1 = result1.drop(index=0, axis=0)

In [ ]:
result1

In [ ]:
result1.to_csv("output/그리드 전처리 완료/서초강남버스아침승차.csv")

## 강남구서초구 퇴근 자전거

In [ ]:
data1 = pd.read_csv("output/서초강남지하철승하차.csv")

In [ ]:
data1

In [ ]:
data1 = data1[["역이름","경도","위도","06시-07시 승차인원","07시-08시 승차인원","08시-09시 승차인원","09시-10시 승차인원"]]

In [ ]:
data1

In [ ]:
data1["아침총승객수"] = data1["06시-07시 승차인원"] + data1["07시-08시 승차인원"]+ data1["08시-09시 승차인원"]+ data1["09시-10시 승차인원"]

In [ ]:
data1 = data1[["역이름","경도","위도","아침총승객수"]]
data1

In [ ]:
data1['sub_cnt']=1
data1['경도'] = data1['경도'].astype(float)
data1['위도'] = data1['위도'].astype(float)
data1['point'] = data1.apply(lambda row : Point([row['경도'], row['위도']]), axis=1)
data1 = gpd.GeoDataFrame(data1, geometry='point')

In [ ]:
data1

In [ ]:
gid_list = []
for i in tqdm(range(data1.shape[0])):
    point = data1['point'].iloc[i]
    if data_gid['geometry'].contains(point).sum() == 0:
        g1 = "Na"
    else :
        g = data_gid['gid'].loc[data_gid['geometry'].contains(point)]
        g1 = g.iloc[0]
    gid_list.append(g1)

In [ ]:
data1['gid'] = gid_list
data1['TL_cnt']=1
data1

In [ ]:
data1 = data1[["gid","아침총승객수"]]
data1

In [ ]:
result1 = data1[['아침총승객수', 'gid']].groupby(['gid']).sum()

In [ ]:
result1

In [ ]:
result1.reset_index(inplace=True)

In [ ]:
result1 = result1.drop(index=0, axis=0)

In [ ]:
result1

In [ ]:
result1.to_csv("output/그리드 전처리 완료/서초강남지하철아침승차.csv")